# Output Stats

## Setup

Import arcpy, pandas, numpy, and osgeo modules. Set the workspace path. Enable overwriting outputs

In [2]:
#import modules
import arcpy
from arcpy.ia import *
import pandas as pd
import numpy as np
from osgeo import gdal

arcpy.env.workspace = 'D:/masters_project/analysis/data' #edit to set work

arcpy.env.overwriteOutput = True #enable output overwrite

## Generate Summary Tables of Ecoregion Cover in Each Scenario

Count the number of cells of each ecoregion in the three scenarios

In [3]:
eco_dict = {'class':[], 'count':[]} #create an empty dictionary
eco = ['historical.tif', 'ssp370_class.tif', 'ssp585_class.tif'] #create a list of present day and predicted scenarios
workspace = 'D:/masters_project/analysis/data' #path as string

for file in eco: #loop for each raster in eco
    path = workspace+'/'+file #path
    raster = gdal.Open(path) #open raster
    band = raster.GetRasterBand(1) #assign band
    cols = raster.RasterXSize #assign columns
    rows = raster.RasterYSize #assign rows
    data = band.ReadAsArray(0, 0, cols, rows).astype(int) #read raster
    uni = np.unique(data,return_counts=True) #find unique values
    eco_dict['class'].append(uni[0]) #add class to dictionary
    eco_dict['count'].append(uni[1]) #add count to dictionary

df_eco = pd.DataFrame(eco_dict) #convert to dataframe
df_eco

,class,count
0,"[0, 1, 2, 3, 4, 5, 6, 7, 8]","[42395, 9242, 5468, 3811, 10918, 3202, 2705, 7..."
1,"[1, 2, 3, 4, 5, 6, 7, 8, 15]","[6165, 2254, 2581, 5453, 5810, 10459, 113, 105..."
2,"[1, 2, 3, 4, 5, 6, 7, 8, 15]","[6430, 1496, 2581, 5175, 5054, 12137, 34, 33, ..."


Clean up the dataframe and save as a .csv

In [4]:
#reorganize df and explode lists
df_eco_exp = (pd.DataFrame({c:df_eco[c].explode() for c in eco_dict})
   .set_index('class', append=True)['count'] 
   .unstack(fill_value=0))

#remove NoData column
pixel_count = df_eco_exp.drop(df_eco_exp.columns[0], axis=1)
pixel_count = pixel_count.drop([15], axis=1)

pixel_count.insert(0, 'rowID', ['historical', 'ssp370', 'ssp585']) #add rowID column
pixel_count.to_csv('D:/masters_project/analysis/output_tables/pixel_counts.csv') #save as a .csv
pixel_count #check values look correct

pixel_count2 = pixel_count.drop(['rowID'], axis=1) #no rowid version for future math

pixel_count #check values look correct

class,rowID,1,2,3,4,5,6,7,8
0,historical,9242,5468,3811,10918,3202,2705,7782,2227
1,ssp370,6165,2254,2581,5453,5810,10459,113,105
2,ssp585,6430,1496,2581,5175,5054,12137,34,33


Calculate % cover for each ecoregion and save as a .csv

In [5]:
#add a new column that sums all the pixels in the row
pixel_count2['pixsum'] = pixel_count2.sum(axis=1)

#create a loop generating new columns for %cover 
i = 0
while i < 8: #loop for columns 1-8
    name = 'class'+str(i+1) #set name for new column, convert to string
    pixel_count2[name] = pixel_count2.iloc[:,i]/pixel_count2.pixsum*100 #append new column calculating %
    i = i+1 #add 1 to i value
    
eco_percent = pixel_count2.drop(pixel_count2.columns[0:9], axis=1) #drop count columns
eco_percent.insert(0, 'rowID', ['historical', 'ssp370', 'ssp585']) #add rowID column

eco_percent.to_csv('D:/masters_project/analysis/output_tables/eco_percent.csv') #save as a .csv
eco_percent #check if it looks right

class,rowID,class1,class2,class3,class4,class5,class6,class7,class8
0,historical,20.377026,12.056003,8.402602,24.072318,7.059861,5.964061,17.157976,4.910153
1,ssp370,18.715847,6.842744,7.835458,16.554341,17.63813,31.75167,0.343048,0.318761
2,ssp585,19.52034,4.541591,7.835458,15.710383,15.343048,36.84578,0.103218,0.100182


Calculate %change in ecoregion cover between each pair of scenarios

In [10]:
pixel_count2 = pixel_count2.drop(pixel_count2.columns[8:17], axis=1) #remove % math from previous chunk

#%change math on rows
change_h370 = (pixel_count2.loc[1,:]-pixel_count2.loc[0,:])/pixel_count2.loc[0,:]*100 #%chng between historical and ssp370
change_h585 = (pixel_count2.loc[2,:]-pixel_count2.loc[0,:])/pixel_count2.loc[0,:]*100 #%chng between historical and ssp585
change_370_585 = (pixel_count2.loc[2,:]-pixel_count2.loc[1,:])/pixel_count2.loc[1,:]*100 #%chng between ssp370 and ssp585 

#add outputs to eco_change df
eco_change = pixel_count2.append(change_h370, ignore_index=True) #add h-370 row to eco% as new df eco_change
eco_change = eco_change.append(change_h585, ignore_index=True) #add h-585 row
eco_change = eco_change.append(change_370_585, ignore_index=True) #add 370-585 row

eco_change = eco_change.replace({np.nan:0}) #replace not a number with 0 again
eco_change2 = eco_change.drop([0, 1, 2]) #drop original cell count rows
eco_change2.insert(0, 'rowID', ['chng_h370', 'chng_h585', 'chng_370_585']) #add rowID column
eco_change2.to_csv('D:/masters_project/analysis/output_tables/eco_change.csv') #save as a .csv

eco_change2

class,rowID,1,2,3,4,5,6,7,8
3,chng_h370,-33.293659,-58.778347,-32.274993,-50.054955,81.449094,286.654344,-98.547931,-95.285137
4,chng_h585,-30.426315,-72.640819,-32.274993,-52.601209,57.838851,348.687616,-99.563094,-98.518186
5,chng_370_585,4.298459,-33.629104,0.000000,-5.098111,-13.012048,16.043599,-69.911504,-68.571429


## Generate Summary Tables For Protected Areas

Redo setup if needed

In [2]:
#import modules
import arcpy
from arcpy.ia import *
import pandas as pd
import numpy as np
from osgeo import gdal, osr

arcpy.env.workspace = 'D:/masters_project/analysis/data' #edit to change workspace pathing

arcpy.env.overwriteOutput = True #enable overwriting saved outputs

Clip all rasters to each protected area boundary. 3 scenarios x 18 pas = 54 output rasters

In [38]:
import arcpy
from arcpy import env
from arcpy.sa import * #import system models again

pa = ['KL_PA.shp', 'Barsey_RS.shp', 'Buxa_TR.shp', 'Chapramari_WS.shp', 'Fambong_Lho_WS.shp', 
      'Gorumara_NP.shp', 'Jigme_Khesar_SNR.shp', 'Jore_Pokhari_SS.shp', 'Kanchenjunga_CA.shp',
      'Khangchendzonga_BR.shp', 'Khitam_BS.shp', 'Kyongnosla_AS.shp', 'Mahananda_WS.shp',
     'Mainam_WS.shp', 'Pangolakha_WS.shp', 'Senchal_WS.shp', 'Singhalila_NP.shp', 'Singhba_RS.shp'] #list protected areas
clip = [] #empty list for outputs
n = 1 #set n to track output numbers

#clip historical data
for boundary in pa: #for each protected area
    x = ExtractByMask('historical.tif', boundary) #clip with extract by mask 
    save = 'D:/masters_project/analysis/data/intermediate/'+'hist_'+str(n)+'.tif' #set path for saving
    x.save(save) #save output to path
    name = 'hist_'+str(n)+'.tif' #set name for storage in clip[]
    clip.append(name) #add clip to list
    n += 1 #increase n for next protected area 

#clip ssp370 data
n = 1 #reset n
for boundary in pa: #for each protected area
    x = ExtractByMask('ssp370_class.tif', boundary) #clip
    save = 'D:/masters_project/analysis/data/intermediate/'+'ssp370_'+str(n)+'.tif' #set path for saving
    x.save(save) #save
    name = 'ssp370_'+str(n)+'.tif' #set raster name
    clip.append(name) #add clip to list
    n += 1 #increase n for next protected area 

#clip ssp585 data
n = 1 #reset n
for boundary in pa: #for each protected area
    x = ExtractByMask('ssp585_class.tif', boundary) #clip
    save = 'D:/masters_project/analysis/data/intermediate/'+'ssp585_'+str(n)+'.tif' #set path for saving
    x.save(save) #save
    name = 'ssp585_'+str(n)+'.tif' #set raster name
    clip.append(name) #add clip to list
    n += 1 #increase n for next protected area 
    
print(clip) #check output - should be 18 rasters for each scenarios

['hist_1.tif', 'hist_2.tif', 'hist_3.tif', 'hist_4.tif', 'hist_5.tif', 'hist_6.tif', 'hist_7.tif', 'hist_8.tif', 'hist_9.tif', 'hist_10.tif', 'hist_11.tif', 'hist_12.tif', 'hist_13.tif', 'hist_14.tif', 'hist_15.tif', 'hist_16.tif', 'hist_17.tif', 'hist_18.tif', 'ssp370_1.tif', 'ssp370_2.tif', 'ssp370_3.tif', 'ssp370_4.tif', 'ssp370_5.tif', 'ssp370_6.tif', 'ssp370_7.tif', 'ssp370_8.tif', 'ssp370_9.tif', 'ssp370_10.tif', 'ssp370_11.tif', 'ssp370_12.tif', 'ssp370_13.tif', 'ssp370_14.tif', 'ssp370_15.tif', 'ssp370_16.tif', 'ssp370_17.tif', 'ssp370_18.tif', 'ssp585_1.tif', 'ssp585_2.tif', 'ssp585_3.tif', 'ssp585_4.tif', 'ssp585_5.tif', 'ssp585_6.tif', 'ssp585_7.tif', 'ssp585_8.tif', 'ssp585_9.tif', 'ssp585_10.tif', 'ssp585_11.tif', 'ssp585_12.tif', 'ssp585_13.tif', 'ssp585_14.tif', 'ssp585_15.tif', 'ssp585_16.tif', 'ssp585_17.tif', 'ssp585_18.tif']


count the number of cells for each class in each clipped raster

In [39]:
workspace = 'D:/masters_project/analysis/data/intermediate' #list workspace as string

pa_dict = {'class':[], 'count':[]} #create an empty dictionary

for file in clip: #list of clipped rasters made earlier
    path = workspace+'/'+file #full path of input raster
    raster = gdal.Open(path) #open raster
    band = raster.GetRasterBand(1) #assign band
    Cols = raster.RasterXSize #assign columns
    Rows = raster.RasterYSize #assign rows
    data = band.ReadAsArray(0, 0, Cols, Rows).astype(int) #read raster
    uni = np.unique(data,return_counts=True) #find unique values
    pa_dict['class'].append(uni[0]) #add class to dictionary
    pa_dict['count'].append(uni[1]) #add count to dictionary

pa_df = pd.DataFrame(pa_dict) #convert to dataframe

pa_df #check output

,class,count
0,"[1, 2, 3, 4, 5, 6, 7, 8, 15]","[4264, 2639, 876, 743, 7, 302, 514, 903, 54327]"
1,"[1, 3, 4, 15]","[2, 105, 136, 450]"
2,"[4, 6, 7, 8, 15]","[55, 188, 182, 903, 2272]"
3,"[7, 15]","[18, 17]"
4,"[4, 15]","[98, 225]"
5,"[7, 15]","[145, 178]"
6,"[1, 2, 3, 4, 15]","[801, 95, 147, 69, 886]"
7,"[4, 15]","[2, 13]"
8,"[1, 2, 3, 4, 15]","[1617, 1237, 206, 128, 1960]"
9,"[1, 2, 3, 4, 5, 15]","[1600, 1313, 183, 110, 1, 3168]"


generate a list of rowID names for output tables 

In [40]:
#generate list for rowID titles
pa = ['All_PA', 'Barsey_RS', 'Buxa_TR', 'Chapramari_WS', 'Fambong_Lho_WS', 
      'Gorumara_NP', 'Jigme_Khesar_SNR', 'Jore_Pokhari_SS', 'Kanchenjunga_CA',
      'Khangchendzonga_BR', 'Khitam_BS', 'Kyongnosla_AS', 'Mahananda',
     'Mainam_WS', 'Pangolakha_WS', 'Senchal_WS', 'Singhalila_NP', 'Singhba_RS'] #list all ecoregion names
raster = ['hist_', 'ssp_370_', 'ssp585_'] #list all scenarios
rowID = [] #empty list to store names

for x in raster: #loop for both scenario and ecoreion list
    for y in pa:
        name = x+y #combine raster and pa name
        rowID.append(name) #add to list

print(rowID) #check output

['hist_All_PA', 'hist_Barsey_RS', 'hist_Buxa_TR', 'hist_Chapramari_WS', 'hist_Fambong_Lho_WS', 'hist_Gorumara_NP', 'hist_Jigme_Khesar_SNR', 'hist_Jore_Pokhari_SS', 'hist_Kanchenjunga_CA', 'hist_Khangchendzonga_BR', 'hist_Khitam_BS', 'hist_Kyongnosla_AS', 'hist_Mahananda', 'hist_Mainam_WS', 'hist_Pangolakha_WS', 'hist_Senchal_WS', 'hist_Singhalila_NP', 'hist_Singhba_RS', 'ssp_370_All_PA', 'ssp_370_Barsey_RS', 'ssp_370_Buxa_TR', 'ssp_370_Chapramari_WS', 'ssp_370_Fambong_Lho_WS', 'ssp_370_Gorumara_NP', 'ssp_370_Jigme_Khesar_SNR', 'ssp_370_Jore_Pokhari_SS', 'ssp_370_Kanchenjunga_CA', 'ssp_370_Khangchendzonga_BR', 'ssp_370_Khitam_BS', 'ssp_370_Kyongnosla_AS', 'ssp_370_Mahananda', 'ssp_370_Mainam_WS', 'ssp_370_Pangolakha_WS', 'ssp_370_Senchal_WS', 'ssp_370_Singhalila_NP', 'ssp_370_Singhba_RS', 'ssp585_All_PA', 'ssp585_Barsey_RS', 'ssp585_Buxa_TR', 'ssp585_Chapramari_WS', 'ssp585_Fambong_Lho_WS', 'ssp585_Gorumara_NP', 'ssp585_Jigme_Khesar_SNR', 'ssp585_Jore_Pokhari_SS', 'ssp585_Kanchenjunga_C

reorganize dataframe and explode

In [41]:
#reorganize df, explode
pa_df_exp = (pd.DataFrame({c:pa_df[c].explode() for c in pa_df})
   .set_index('class', append=True)['count'] 
   .unstack(fill_value=0))

#remove NoData column
pa_pixel_count = pa_df_exp.drop(pa_df_exp.columns[0], axis=1)
pa_pixel_count = pa_pixel_count.drop([15], axis=1)

pa_pixel_count.insert(0, 'rowID', rowID) #add rowID column
pa_pixel_count.to_csv('D:/masters_project/analysis/output_tables/pa_pixel_counts.csv') #save as a .csv

pa_pixel_count2 = pa_pixel_count.drop(['rowID'], axis=1) #no rowid version for future math

pa_pixel_count #check values look correct

class,rowID,1,2,3,4,5,6,7,8
0,hist_All_PA,4264,2639,876,743,7,302,514,903
1,hist_Barsey_RS,2,0,105,136,0,0,0,0
2,hist_Buxa_TR,0,0,0,55,0,188,182,903
3,hist_Chapramari_WS,0,0,0,0,0,0,18,0
4,hist_Fambong_Lho_WS,0,0,0,98,0,0,0,0
5,hist_Gorumara_NP,0,0,0,0,0,0,145,0
6,hist_Jigme_Khesar_SNR,801,95,147,69,0,0,0,0
7,hist_Jore_Pokhari_SS,0,0,0,2,0,0,0,0
8,hist_Kanchenjunga_CA,1617,1237,206,128,0,0,0,0
9,hist_Khangchendzonga_BR,1600,1313,183,110,1,0,0,0


Calculate %cover of ecoregions in each row

In [42]:
#add a new column that sums all the pixels in the row
pa_pixel_count2['pixsum'] = pa_pixel_count2.sum(axis=1)

#create a loop generating new columns for 
i = 0
while i < 8: #loop for columns 1-8
    name = 'class'+str(i+1) #set name for new column, convert to string
    pa_pixel_count2[name] = pa_pixel_count2.iloc[:,i]/pa_pixel_count2.pixsum*100 #append new column calculating %
    i = i+1 #add 1 to i value
pa_eco_percent = pa_pixel_count2.drop(pa_pixel_count2.columns[0:9], axis=1) #drop count columns

pa_eco_percent.insert(0, 'rowID', rowID) #add rowID column
pa_eco_percent.to_csv('D:/masters_project/analysis/output_tables/pa_eco_percent.csv') #save as a .csv
pa_eco_percent #check if it looks right

class,rowID,class1,class2,class3,class4,class5,class6,class7,class8
0,hist_All_PA,41.608119,25.751366,8.548009,7.250195,0.068306,2.946916,5.015613,8.811475
1,hist_Barsey_RS,0.823045,0.0,43.209877,55.967078,0.0,0.0,0.0,0.0
2,hist_Buxa_TR,0.0,0.0,0.0,4.141566,0.0,14.156627,13.704819,67.996988
3,hist_Chapramari_WS,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0
4,hist_Fambong_Lho_WS,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0
5,hist_Gorumara_NP,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0
6,hist_Jigme_Khesar_SNR,72.032374,8.543165,13.219424,6.205036,0.0,0.0,0.0,0.0
7,hist_Jore_Pokhari_SS,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0
8,hist_Kanchenjunga_CA,50.721455,38.801757,6.461731,4.015056,0.0,0.0,0.0,0.0
9,hist_Khangchendzonga_BR,49.890864,40.94169,5.706268,3.429997,0.031182,0.0,0.0,0.0


Calculate %change between each pair of scenarios in each protected area

In [7]:
count = pd.read_csv('D:/masters_project/analysis/output_tables/pa_pixel_counts.csv') #call pixel count csv from before
count = count.drop(['rowID', 'Unnamed: 0'], axis=1) #drop null rows
count = count.T #transpose table

#generate names for rowIDs
pa = ['All_PA', 'Barsey_RS', 'Buxa_TR', 'Chapramari_WS', 'Fambong_Lho_WS', 
      'Gorumara_NP', 'Jigme_Khesar_SNR', 'Jore_Pokhari_SS', 'Kanchenjunga_CA',
      'Khangchendzonga_BR', 'Khitam_BS', 'Kyongnosla_AS', 'Mahananda',
     'Mainam_WS', 'Pangolakha_WS', 'Senchal_WS', 'Singhalila_NP', 'Singhba_RS'] #protected area names
change = ['h370_', 'h585_', '370_585_'] #%change categories
rowID = [] #empty list for rowid names

for x in change: #loop
    for y in pa:
        name = 'chng_'+x+y #combine each %change category with a pa
        rowID.append(name) #add rowID to list

#calculate %change between historical and ssp370 scenarios
i = 0
while i < 18: #loop for columns 0-17 (hist)
    name = rowID[i] #set column name as rowID
    j = i+18 #ssp370 is 18 rows beyond historical
    count[name] = (count.iloc[:, j]-count.iloc[:, i])/count.iloc[:, i]*100 #append new column calculating %chng
    i = i+1 #add 1 to i value

#calculate %change between historical and ssp585 scenarios
i = 0
while i < 18: #loop for columns 0-17 (hist)
    name = rowID[i+18] #set column name as rowID
    j = i+36 #ssp585 is 36 rows beyond historical
    count[name] = (count.iloc[:, j]-count.iloc[:, i])/count.iloc[:, i]*100 #append new column calculating %chng
    i = i+1 #add 1 to i value

#calculate %change between ssp370 and ssp585 scenarios
while i < 36: #loop for columns 18-35 (ssp370)
    name = rowID[i+18] #set column name as rowID
    j = i+18 #ssp585 is 18 rows beyond ssp370
    count[name] = (count.iloc[:, j]-count.iloc[:, i])/count.iloc[:, i]*100 #append new column calculating %chng
    i = i+1 #add 1 to i value

pa_eco_change = count.drop(count.columns[0:54], axis=1) #drop count columns
pa_eco_change = pa_eco_change.replace({np.nan:0}) #replace NaN from div by 0 errors with 0
pa_eco_change = pa_eco_change.replace({np.inf:100}) #replace inf from div by 0 errors with 100
pa_eco_change = pa_eco_change.T #transpose dataframe back

pa_eco_change.to_csv('D:/masters_project/analysis/output_tables/pa_eco_change.csv') #save to .csv

pa_eco_change #check outputs

,1,2,3,4,5,6,7,8
chng_h370_All_PA,-24.554409,-61.765820,-10.616438,26.379542,3028.571429,314.238411,-100.0,-100.0
chng_h370_Barsey_RS,-100.000000,0.000000,-89.523810,10.294118,100.000000,0.000000,0.0,0.0
chng_h370_Buxa_TR,0.000000,0.000000,0.000000,-98.181818,0.000000,411.170213,-100.0,-100.0
chng_h370_Chapramari_WS,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,-100.0,0.0
chng_h370_Fambong_Lho_WS,0.000000,0.000000,0.000000,-92.857143,100.000000,0.000000,0.0,0.0
chng_h370_Gorumara_NP,0.000000,0.000000,0.000000,0.000000,100.000000,100.000000,-100.0,0.0
chng_h370_Jigme_Khesar_SNR,-43.945069,-100.000000,21.768707,143.478261,100.000000,0.000000,0.0,0.0
chng_h370_Jore_Pokhari_SS,0.000000,0.000000,0.000000,-50.000000,0.000000,0.000000,0.0,0.0
chng_h370_Kanchenjunga_CA,-15.151515,-62.974939,28.155340,27.343750,100.000000,0.000000,0.0,0.0
chng_h370_Khangchendzonga_BR,-22.062500,-56.283321,30.601093,113.636364,2800.000000,0.000000,0.0,0.0
